#### Задание 1 
Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.

In [1]:
import pymongo

In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
db = client['vacancy_database_2']

In [5]:
collection = db.vacancy

In [6]:
from bs4 import BeautifulSoup
import lxml
import requests

In [7]:
header = {
    'USER-AGENT': \
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

In [8]:
for i in range(1, 3):
    url = "https://krasnoyarsk.hh.ru/search/vacancy?page={}".format(i)
    response_hh = requests.get(url, headers=header)
    soup_hh = BeautifulSoup(response_hh.text, 'lxml')
    vacancy_name = [i.text.strip() for i in soup_hh.find_all('a', {'data-qa':'vacancy-serp__vacancy-title'})]
    vacancy_html = [i['href'] for i in soup_hh.find_all('a', {'data-qa':'vacancy-serp__vacancy-title'})]
    for num, i in enumerate(vacancy_name):
        collection.insert_one(
            {'Name': i,
             'Income': None,
             'Html': vacancy_html[num],
             'Site': 'HH.ru'})

In [9]:
import unicodedata

In [10]:
for i in range(1, 3):
    url2 = "https://krasnoyarsk.superjob.ru/vacancy/search/?page={}".format(i)
    response_sj = requests.get(url2, headers=header)
    soup_sj = BeautifulSoup(response_sj.text, 'lxml')
    vacancy_name2 = [i.text.strip() for i in soup_sj.find_all(class_='_6AfZ9')]
    vacancy_price2 = [i.text.strip() for i in soup_sj.find_all(class_='_1h3Zg _2Wp8I _2rfUm _2hCDz _2ZsgW')]
    vacancy_price_new2 =[]
    for i in vacancy_price2:
        i = unicodedata.normalize("NFKD", i)
        vacancy_price_new2.append(i)
    vacancy_html2 = [i['href'] for i in soup_sj.find_all(class_='_6AfZ9')]
    for num, i in enumerate(vacancy_name2):
        collection.insert_one(
            {'Name': i,
             'Income': vacancy_price_new2[num],
             'Html': vacancy_html[num],
             'Site': 'SJ.ru'})

#### Задание 2  
Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.

Не разобрался как выделить максимальную и минимальную зарплату во втором задании, а оно пока не проверено

In [11]:
def much_income(number):
    collection.find({'Income':{'$lt': number}})

#### Задание 3  
Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

Для выполнения задания в качестве примера сделал функцию, которая добавляет новые вакансии с сайта HH.ru

In [12]:
def new_vacancy(pages):
    for i in range(0, pages):
        url = "https://krasnoyarsk.hh.ru/search/vacancy?page={}".format(i)
        response_hh = requests.get(url, headers=header)
        soup_hh = BeautifulSoup(response_hh.text, 'lxml')
        vacancy_name = [i.text.strip() for i in soup_hh.find_all('a', {'data-qa':'vacancy-serp__vacancy-title'})]
        vacancy_html = [i['href'] for i in soup_hh.find_all('a', {'data-qa':'vacancy-serp__vacancy-title'})]
        for num, i in enumerate(vacancy_name):
            collection.replace_one(
                {'Html': vacancy_html[num]},
                {'Name': i,
                 'Income': None,
                 'Html': vacancy_html[num],
                 'Site': 'HH.ru'},
                upsert = True)

In [13]:
new_vacancy(2)